In [1]:
%%html
<style>
.output_subarea.output_text.output_stream.output_stdout > pre {
    width:max-content;
}
.p-Widget.jp-RenderedText.jp-OutputArea-output > pre {
   width:max-content;
}
</style>

In [2]:
import sys
import os
import operator

from pyspark.sql import Row
from pyspark.sql.functions import udf, when, rand, struct, col, lit, current_date, current_timestamp, monotonically_increasing_id
from pyspark.sql.types import StringType, IntegerType, StructField, StructType, DoubleType, ArrayType, FloatType

In [3]:
import visualize
from ophelia.ophelib.OpheliaMain import Ophelia

In [ ]:
module_path = os.path.abspath(os.path.join('..'))

In [5]:
ophelia = Ophelia("Risk Classification")
sc = ophelia.Spark.build_spark_context()
#sc.addFile(module_path+"/enquire/", recursive = True)


22:04:06.037 Ophelia [INFO] My name is Ophelia Vendata
22:04:06.037 Ophelia [INFO] I am an artificial assistant for data mining & ML engine with spark
22:04:06.037 Ophelia [INFO] Welcome to Ophelia spark miner engine
22:04:06.037 Ophelia [INFO] Lib Version Ophelia.dev1.0
22:04:06.037 Ophelia [WARN] V for Vendata...

22:04:06.037 Ophelia [WARN] Initializing Spark Session
22:04:06.040 Ophelia [INFO] Spark Version: 3.0.0
22:04:06.040 Ophelia [INFO] This Is: 'Risk Classification' App
22:04:06.040 Ophelia [INFO] Spark Context Initialized Success


In [9]:
spark = ophelia.SparkSession
path = "data/raw/csv/bank.csv"
customer_data = spark.read.csv(path, header=True, inferSchema=True)
customer_data.printSchema()
customer_data.show(5, False)

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)

+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|job       |marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+----------+-------+---------+-------+-------+-------+----+-------+--

In [10]:
column_selection = [
    col('age').cast('int').alias('age'),
    col('job').cast('string').alias('job'),
    col('marital').cast('string').alias('marital'),
    col('education').cast('string').alias('education'),
    col('gender').cast('string').alias('gender'),
    col('child').cast('float').alias('child'),
    col('saving').cast('float').alias('saving'),
    col('insight').cast('float').alias('insight'),
    col('backup').cast('float').alias('backup')
]

struct_assembler = customer_data.select(*column_selection, struct(column_selection).alias("struct_assembler"))
struct_assembler.show(5, False)
struct_assembler.printSchema()

AnalysisException: cannot resolve '`gender`' given input columns: [age, balance, campaign, contact, day, default, deposit, duration, education, housing, job, loan, marital, month, pdays, poutcome, previous];;
'Project [cast(age#288 as int) AS age#408, cast(job#289 as string) AS job#409, cast(marital#290 as string) AS marital#410, cast(education#291 as string) AS education#411, cast('gender as string) AS gender#412, cast('child as float) AS child#413, cast('saving as float) AS saving#414, cast('insight as float) AS insight#415, cast('backup as float) AS backup#416, struct(age, cast(age#288 as int), job, cast(job#289 as string), marital, cast(marital#290 as string), education, cast(education#291 as string), NamePlaceholder, cast('gender as string), NamePlaceholder, cast('child as float), NamePlaceholder, cast('saving as float), NamePlaceholder, cast('insight as float), NamePlaceholder, cast('backup as float)) AS struct_assembler#417]
+- Relation[age#288,job#289,marital#290,education#291,

In [ ]:
def build_tree_classifier():
    schema_tree = ArrayType(StructType([
        StructField("vote", FloatType(), False),
        StructField("weight", FloatType(), False),
        StructField("risk_label", StringType(), False)
    ]))
    return udf(ophelia.opClassify.tree_generator, schema_tree)

def build_risk_class():
    return udf(ophelia.opClassify.run_classification_risk, StringType())

In [ ]:
tree_model_udf = build_tree_classifier()
risk_class_udf = build_risk_class()

In [ ]:
tree_model_df = struct_assembler.select("*", tree_model_udf(col("struct_assembler")).alias("tree")).cache()
tree_model_df.show(5, False)
tree_model_df.printSchema()

In [ ]:
risk_class_df = tree_model_df.select(risk_class_udf(col("tree")).alias("risk_label"),
                                     *column_selection, current_date().alias("information_date"),
                                     current_timestamp().alias("model_date"))

risk_class_df.groupBy("risk_label").count().show(10, False)
risk_class_df.show(10, False)
risk_class_df.printSchema()

In [ ]:
columns_risk = risk_class_df.columns
when_conditions = (
    when(col("risk_label") == "A", lit(5.0)).otherwise(
        when(col("risk_label") == "MA", lit(4.0)).otherwise(
            when(col("risk_label") == "M", lit(3.0)).otherwise(
                when(col("risk_label") == "MC", lit(2.0)).otherwise(
                    when(col("risk_label") == "C", lit(1.0))))))
)
mapped_risk_label = risk_class_df.select((monotonically_increasing_id() + 100000000).alias("customer_id"), 
                                         when_conditions.alias("risk_label_id"), *columns_risk)
mapped_risk_label.groupBy('risk_label_id', 'risk_label').count().orderBy('risk_label_id').show()
mapped_risk_label.show(5, False)

In [ ]:
riskClassPath = ophelia.opWrite.write_parquet(
    df=mapped_risk_label,
    output_type="model",
    project="RiskClassifier",
    part="information_date"
)